In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
URL_BASE = "https://pt.wikipedia.org/"
PAGINA_INICIAL = "wiki/Wikip%C3%A9dia:P%C3%A1gina_principal"

resposta = requests.get(URL_BASE + PAGINA_INICIAL)
soup = BeautifulSoup(resposta.content, "html.parser")

In [ ]:
soup.title

In [ ]:
import time

for i in range(1):
  resposta = requests.get(URL_BASE + PAGINA_INICIAL)
  print(f"Chamada {i}: Obtive pagina {URL_BASE + PAGINA_INICIAL}")
  time.sleep(1)

Chamada 0: Obtive pagina https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal


In [ ]:
# Salvando página

url = "https://pt.wikipedia.org/wiki/Mariah_Carey"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Vitória 64; x64; rv:142.0) Gecko/20100101 Firefox/142.0"
}

resposta = requests.get(url, headers=headers)
soup = BeautifulSoup(resposta.content, "html.parser")

# Obtem o titulo da pagina
titulo = soup.find("span", {"class": "mw-page-title-main"})
titulo = titulo.text
print(titulo)

# Salva a página em um arquivo
with open(f"{titulo}.html", "w", encoding="utf-8") as arquivo:
  conteudo = resposta.content.decode("utf-8")
  arquivo.write(conteudo)

Mariah Carey


In [ ]:
# lendo o conteúdo de uma página
with open(f"{titulo}.html", "r", encoding="utf-8") as arquivo:
  conteudo = ""
  for linha in arquivo.readlines():
    conteudo += linha


In [ ]:
import re
import requests
import time
from bs4 import BeautifulSoup
from collections import deque

URL_BASE = "https://pt.wikipedia.org"
HEADERS = {"User-Agent": "Mozilla/5.0"}
MAX_PESSOAS = 1000

# Fila de links a visitar (fila)
fila_links = deque()
visitados = set()  # Para evitar visitar a mesma página várias vezes
link_pessoas = []

# Inicializa com a página principal
pagina_inicial = URL_BASE + "/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal"
fila_links.append(pagina_inicial)
visitados.add(pagina_inicial)

contador_paginas = 0

while fila_links and len(link_pessoas) < MAX_PESSOAS:
    pagina_atual = fila_links.popleft()
    contador_paginas += 1

    # Atraso para evitar sobrecarregar o servidor
    time.sleep(0.5)

    print(f"Acessando {pagina_atual} ({contador_paginas})")

    try:
        # Adicionei um timeout para evitar que o script trave em requisições lentas
        response = requests.get(pagina_atual, headers=HEADERS, timeout=10)
        # Verifica se a resposta HTTP é 200 (sucesso)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
    except requests.exceptions.RequestException as e:
        print(f"Erro de requisição para {pagina_atual}: {e}")
        continue
    except Exception as e:
        print(f"Erro inesperado em {pagina_atual}: {e}")
        continue

    # Extrair infobox
    infobox = soup.find("table", {"class": "infobox"})
    if infobox:
        # A busca por "Nascimento" ou "Nascido" é mais precisa para perfis
        tags = [tag.get_text(strip=True) for tag in infobox.find_all("th", {"class": "infobox-label"})]
        if "Nascimento" in tags or "Nascido em" in tags:
            # Garante que não é uma página já encontrada
            if pagina_atual not in link_pessoas:
                link_pessoas.append(pagina_atual)
                print(f"Pessoa encontrada! Total: {len(link_pessoas)}")

    # Extrair links desta página e adicionar à fila
    # O filtro regex é crucial aqui para evitar links externos ou de metadados
    for link in soup.find_all("a", href=re.compile("^(/wiki/)((?!:).)*$")):
        href = link.get("href")
        if href:
            url_completa = URL_BASE + href
            if url_completa not in visitados:
                fila_links.append(url_completa)
                visitados.add(url_completa)

print(f"--- FIM DA EXECUÇÃO ---")
print(f"Processados {contador_paginas} links. Foram encontradas {len(link_pessoas)} pessoas.")
print("Links das pessoas encontradas:")
for pessoa_link in link_pessoas:
    print(pessoa_link)

Acessando https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal (1)
Acessando https://pt.wikipedia.org/wiki/Wikip%C3%A9dia (2)
Acessando https://pt.wikipedia.org/wiki/Enciclop%C3%A9dia (3)
Acessando https://pt.wikipedia.org/wiki/Conte%C3%BAdo_livre (4)
Acessando https://pt.wikipedia.org/wiki/L%C3%ADngua_portuguesa (5)
Acessando https://pt.wikipedia.org/wiki/Oops!..._I_Did_It_Again (6)
Acessando https://pt.wikipedia.org/wiki/%C3%81lbum_de_est%C3%BAdio (7)
Acessando https://pt.wikipedia.org/wiki/Britney_Spears (8)
Pessoa encontrada! Total: 1
Acessando https://pt.wikipedia.org/wiki/Jive_Records (9)
Acessando https://pt.wikipedia.org/wiki/...Baby_One_More_Time (10)
Acessando https://pt.wikipedia.org/wiki/...Baby_One_More_Time_Tour (11)
Acessando https://pt.wikipedia.org/wiki/Produtores_musicais (12)
Acessando https://pt.wikipedia.org/wiki/Max_Martin (13)
Acessando https://pt.wikipedia.org/wiki/Rami_Yacoub (14)
Pessoa encontrada! Total: 2
Acessando https://pt.wikipedia.org/wiki

KeyboardInterrupt: 